In [ ]:
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from __future__ import division
from IPython.core.display import Image
#from deltasigma import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# -*- coding: utf-8 -*-
# _synthesizeNTF.py
# Module providing the synthesizeNTF function
# Copyright 2013 Giuseppe Venturini
# This file is distributed with python-deltasigma.
#
# python-deltasigma is a 1:1 Python replacement of Richard Schreier's
# MATLAB delta sigma toolbox (aka "delsigma"), upon which it is heavily based.
# The delta sigma toolbox is (c) 2009, Richard Schreier.
#
# python-deltasigma is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# LICENSE file for the licensing terms.
#
# The following code has been (slightly) modified from pydsm, its original
# copyright notice follows:
#
# Copyright (c) 2012, Sergio Callegari
# All rights reserved.
#
# The py dsm code was ported from the MATLAB Delta Sigma toolbox
# Copyright (c) 2009, Richard Schreier
#
# The three software follow the same license, known as the 2-clause BSD.
# See the LICENSE file for details.

"""
Module providing the main NTF synthesis function.
"""

from warnings import warn

import numpy as np

from _config import optimize_NTF
from _synthesizeNTF0 import synthesizeNTF0
from _synthesizeNTF1 import synthesizeNTF1


def synthesizeNTF(order=3, osr=64, opt=0, H_inf=1.5, f0=0.0):
    """Synthesize a noise transfer function for a delta-sigma modulator.

    **Parameters:**

    order : int, optional
        the order of the modulator, defaults to 3
    osr : float, optional
        the oversampling ratio, defaults to 64
    opt : int or list of floats, optional
        flag for optimized zeros, defaults to 0

        * 0 -> not optimized,
        * 1 -> optimized,
        * 2 -> optimized with at least one zero at band-center,
        * 3 -> optimized zeros (with optimizer)
        * 4 -> same as 3, but with at least one zero at band-center
        * [z] -> zero locations in complex form

    H_inf : real, optional
        max allowed peak value of the NTF. Defaults to 1.5
    f0 : real, optional
        center frequency for BP modulators, or 0 for LP modulators.
        Defaults to 0.
        1 corresponds to the sampling frequency, so that 0.5 is the
        maximum value. Value 0 specifies an LP modulator.

    **Returns:**

    ntf : tuple
        noise transfer function in zpk form.

    **Raises:**

    ValueError
        * 'Error. f0 must be less than 0.5' if f0 is out of range

        * 'Order must be even for a bandpass modulator.' if the order is
          incompatible with the modulator type.

        * 'The opt vector must be of length xxx' if opt is used to explicitly
          pass the NTF zeros and these are in the wrong number.

    **Warns:**

        * 'Creating a lowpass ntf.' if the center frequency is different
          from zero, but so low that a low pass modulator must be designed.

        * 'Unable to achieve specified H_inf ...' if the desired H_inf
          cannot be achieved.

        * 'Iteration limit exceeded' if the routine converges too slowly.

    **Notes:**

    This is actually a wrapper function which calls the appropriate version
    of synthesizeNTF, based on the module control flag `optimize_NTF` which
    determines whether to use optimization tools.

    Parameter ``H_inf`` is used to enforce the Lee stability criterion.

    **Example:**

    Fifth-order lowpass modulator; zeros optimized for an oversampling ratio of 32.::

        from deltasigma import *
        H = synthesizeNTF(5, 32, 1)
        pretty_lti(H)

    Returns::

              (z -1) (z^2 -1.997z +1) (z^2 -1.992z +0.9999)
        --------------------------------------------------------
         (z -0.7778) (z^2 -1.796z +0.8549) (z^2 -1.613z +0.665)

    .. plot::

        from deltasigma import *
        H = synthesizeNTF(5, 32, 1)
        DocumentNTF(H, 32)

    .. seealso::

       :func:`clans` : Closed-Loop Analysis of Noise-Shaper.
               An alternative method for selecting NTFs based on the 1-norm of the
               impulse response of the NTF

       :func:`synthesizeChebyshevNTF()` : Select a type-2 highpass Chebyshev NTF.
           This function does a better job than synthesizeNTF if osr
           or H_inf is low.

    """
    if f0 > 0.5:
        raise ValueError('Error. f0 must be less than 0.5.')
    if f0 != 0 and f0 < 0.25/osr:
        warn('Creating a lowpass ntf.')
        f0 = 0
    if f0 != 0 and order % 2 != 0:
        raise ValueError('Order must be even for a bandpass modulator.')
    opt = np.asarray(opt)
    if opt.ndim > 1 or (opt.ndim == 1 and opt.size != order):
        raise ValueError('The opt vector must be of length %d.' % order)

    if not optimize_NTF:
        ntf = synthesizeNTF0(order, osr, opt, H_inf, f0)
    else:
        ntf = synthesizeNTF1(order, osr, opt, H_inf, f0)
    return ntf


In [ ]:
alpha = (-2, 2, 0.1)
H = synthesizeNTF(2,64)
print(H)
#pretty_lti(H)

In [ ]:
ntf1 = (np.array([1. + 0.j, 1. + 0.j]), np.array([0. + 0.j, 0. + 0.j]), 1)
print(ntf1)

In [ ]:
# -*- coding: utf-8 -*-
# _simulateSNR.py
# Module providing the simulateSNR function
# Copyright 2013 Giuseppe Venturini
# This file is part of python-deltasigma.
#
# python-deltasigma is a 1:1 Python replacement of Richard Schreier's
# MATLAB delta sigma toolbox (aka "delsigma"), upon which it is heavily based.
# The delta sigma toolbox is (c) 2009, Richard Schreier.
#
# python-deltasigma is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# LICENSE file for the licensing terms.

"""Module providing the simulateSNR() function
"""

from __future__ import division, print_function

import collections
from warnings import warn

import numpy as np
from numpy.fft import fft, fftshift

from _calculateSNR import calculateSNR
from _mapQtoR import mapQtoR
from _simulateDSM import simulateDSM
from _simulateQDSM import simulateQDSM
from _utils import _get_zpk


def simulateSNR(arg1, osr, amp=None, f0=0, nlev=16, f=None, k=13,
                quadrature=False):
    """Determine the SNR for a delta-sigma modulator by using simulations.

    Simulate a delta-sigma modulator with sine wave inputs of various
    amplitudes and calculate the signal-to-noise ratio (SNR) in dB for each 
    input.

    Three alternative descriptions of the modulator can be used:

     * The modulator is described by a noise transfer function (NTF), provided
       as ``arg1`` and the number of quantizer levels (``nlev``).

     * Alternatively, the first argument to simulateSNR may be an ABCD matrix. 

     * Lastly, ``arg1`` may be a function taking the input signal as its
       sole argument.

    The band of interest is defined by the oversampling ratio (``osr``)
    and the center frequency (``f0``).

    The input signal is characterized by the ``amp`` vector and the ``f`` variable.
    A default value for ``amp`` is used if not supplied.

    ``f`` is the input frequency, normalized such that 1 -> fs;
    ``f`` is rounded to an FFT bin.

    Using sine waves located in FFT bins, the SNR is calculated as the ratio
    of the sine wave power to the power in all in-band bins other than those
    associated with the input tone. Due to spectral smearing, the input tone
    is not allowed to lie in bins 0 or 1. The length of the FFT is :math:`2^k`.

    If the NTF is complex, :func:`simulateQDSM` (which is slow, also available
    in a future release) is called.

    If ABCD is complex, :func:`simulateDSM` is used with the real equivalent
    of ABCD in order to speed up simulations.

    Future versions may accommodate STFs.

    **Parameters:**

    arg1 : scipy 'lti' object, or ndarray
        The first argument may be one of the various supported representations
        for an NTF or an ABCD matrix. Quadrature modulators are supported both
        with quadrature TFs or with quadrature ABCD matrices, the latter being
        the recommended description.

    osr : int
        The over-sampling ratio.

    amp : sequence, optional
        The amplitudes in dB, referred to the FS, for which the SNR is to be 
        evaluated. ``amp`` defaults to [-120 -110...-20 -15 -10 -9 -8 ... 0]dB,
        where 0 dB means a full-scale (peak value = nlev-1) sine wave.

    f0 : float, optional
        The center frequency. Normalized. Defaults to 0.

    nlev : int, optional
        Number of quantizer levels, defaults to 2.

    f : float, optional
        Test signal input frequency. Normalized. Rounded to an FFT bin.
        Defaults to:

    .. math::

        f = \\frac{1}{4\ \mathrm{OSR}}

    k : int, optional
        The number of samples used to compute the FFT is set by the integer `k`
        - default value 13 - through:

    .. math::

        N_{samples} = 2^k

    quadrature : boolean, optional
        Whether the delta sigma modulator is a quadrature modulator or not.
        Defaults to ``False``.

    **Returns:**

    snr : ndarray
        The SNR, from simulation.

    amp : ndarray
        The amplitudes corresponding to the SNR values.

    .. rubric:: Example:

    Compare the SNR vs input amplitude curve for a fifth-order modulator, as 
    determined by the describing function method (:func:`predictSNR`) with
    that determined by simulation (:func:`simulateSNR`).::

        import pylab as plt
        from deltasigma import *
        OSR = 32
        H = synthesizeNTF(5, OSR, 1)
        snr_pred, amp, _, _, _ = predictSNR(H,OSR)
        snr, amp = simulateSNR(H, OSR)
        plt.plot(amp, snr_pred, 'b', label='Predicted')
        plt.hold(True)
        plt.plot(amp, snr, 'go', label='Simulated')
        plt.grid(True)
        figureMagic([-100, 0], 10, None,
                    [0, 100], 10, None)
        plt.xlabel('Input Level, dB')
        plt.ylabel('SNR, dB')
        s = 'peak SNR = %4.1fdB\\n' % max(snr)
        plt.text(-65, 15, s, horizontalalignment='left')
        plt.legend(loc='best')

    .. plot::

        import pylab as plt
        from deltasigma import *
        OSR = 32
        H = synthesizeNTF(5, OSR, 1)
        snr_pred, amp, _, _, _ = predictSNR(H,OSR)
        snr, amp = simulateSNR(H, OSR)
        plt.plot(amp, snr_pred, 'b', label='Predicted')
        plt.hold(True)
        plt.plot(amp, snr, 'go', label='Simulated')
        plt.grid(True)
        figureMagic([-100, 0], 10, None,
                    [0, 100], 10, None)
        plt.xlabel('Input Level, dB')
        plt.ylabel('SNR, dB')
        s = 'peak SNR = %4.1fdB\\n' % max(snr)
        plt.text(-65, 15, s, horizontalalignment='left')
        plt.legend(loc='best')

    """
    # Look at arg1 and decide if the system is quadrature
    quadrature_ntf = False
    if callable(arg1):
        raise ValueError('There is no support for NTFs described through ' +
                         'a function.')
    elif not isinstance(arg1, np.ndarray):
        # NTF of LTI type or zpk tuple, or (num,den) etc...
        # in this case, if the modulator is in quadrature, 
        # we will have to call simulateQDSM
        for roots in _get_zpk(arg1)[:2]:
            if np.any(np.abs(np.imag(np.poly(roots))) > 0.0001):
                quadrature = True
                quadrature_ntf = True
    else: # ABCD matrix, no matter what, here we will get through the
        # simulation with simulateDSM()
        arg1 = np.real_if_close(arg1) # tiny imaginary parts due to rounding
        if not np.all(np.imag(arg1) == 0):
            quadrature = True

    if amp is None:
        amp = np.concatenate((np.arange(- 120, -20 + 1, 10),
                              np.array((-15,)),
                              np.arange(-10, 1)))
    elif not isinstance(amp, collections.Iterable):
        amp = np.array((amp, ))
    else:
        amp = np.asarray(amp)
    osr_mult = 2
    if f0 != 0 and not quadrature:
        osr_mult = 2*osr_mult
    if f is None or np.isnan(f):
        f = f0 + 0.5/(osr*osr_mult) # Halfway across the band
    M = nlev - 1
    if quadrature and not quadrature_ntf:
        # Modify arg1 (ABCD) and nlev so that simulateDSM can be used
        nlev = np.array([nlev, nlev])
        arg1 = mapQtoR(arg1)
    if abs(f - f0) > 1./(osr*osr_mult):
        warn('The input tone is out-of-band.')
    N = 2**k
    if N < 8*2*osr:
        warn('Increasing k to accommodate a large oversampling ratio.')
        k = np.array(np.ceil(np.log2(8*2*osr)), dtype=np.int64)
        N = 2**k
    F = np.round(f*N)
    if np.abs(F) <= 1:
        warn('Increasing k to accommodate a low input frequency.')
        # Want f*N >= 1
        k = np.ceil(np.log2(1./f))
        N = 2**k
        F = 2
    Ntransient = 100
    soft_start = 0.5*(1 - np.cos(2*np.pi/Ntransient * \
                                 np.arange(Ntransient/2)))
    if not quadrature:
        tone = M*np.sin(2*np.pi*F/N*np.arange(N + Ntransient))
        tone[:Ntransient//2] = tone[:Ntransient//2] * soft_start
    else:
        tone = M*np.exp(2j*np.pi*F/N * np.arange(N + Ntransient))
        tone[:Ntransient//2] = tone[:Ntransient//2] * soft_start
        if not quadrature_ntf:
            tone = tone.reshape((1, -1))
            tone = np.vstack((np.real(tone), np.imag(tone)))
    # create a Hann window
    window = 0.5*(1 - np.cos(2*np.pi*np.arange(N)/N))
    if f0 == 0:
        # Exclude DC and its adjacent bin
        inBandBins = int(N/2) + np.arange(3,
                                     np.round(N/osr_mult/osr) + 1,
                                     dtype=np.int32)
        F = F - 2
    else:
        f1 = np.round(N*(f0 - 1./osr_mult/osr))
        # Should exclude DC
        inBandBins = int(N/2) + np.arange(f1,
                                     np.round(N*(f0 + 1./osr_mult/osr)) + 1,
                                     dtype=np.int32)
        F = F - f1 + 1
    snr = np.zeros(amp.shape)
    for i, A in enumerate(np.power(10.0, amp/20)):
        if quadrature_ntf:
            v, _, _, _ = simulateQDSM(A*tone, arg1, nlev)
        else:
            v, _, _, _ = simulateDSM(A*tone, arg1, nlev)
            if quadrature:
                v = v[0, :] + 1j*v[1, :]
        hwfft = fftshift(fft(window*v[Ntransient:N + Ntransient]))
        snr[i] = calculateSNR(hwfft[inBandBins - 1], F)
    return snr, amp



In [ ]:
simulateSNR(ntf1, osr = 8)

In [ ]:
# -*- coding: utf-8 -*-
# _DocumentNTF.py
# Module providing DocumentNTF
# Copyright 2013 Giuseppe Venturini
# This file is part of python-deltasigma.
#
# python-deltasigma is a 1:1 Python replacement of Richard Schreier's
# MATLAB delta sigma toolbox (aka "delsigma"), upon which it is heavily based.
# The delta sigma toolbox is (c) 2009, Richard Schreier.
#
# python-deltasigma is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# LICENSE file for the licensing terms.

"""Module providing the DocumentNTF function.
"""

from __future__ import division

import numpy as np
import pylab as plt

from _calculateTF import calculateTF
from _dbv import dbv
from _ds_f1f2 import ds_f1f2
from _ds_freq import ds_freq
from _evalTF import evalTF
from _figureMagic import figureMagic
from _infnorm import infnorm
from _plotPZ import plotPZ
from _rmsGain import rmsGain


def DocumentNTF(arg1, osr=64, f0=0, quadrature=False):
    """Plot the NTF's poles and zeros as well as its frequency-response

    The first argument is either the NTF or ABCD matrix.
    If the first argument is ABCD, the STF is also plotted.

    .. plot::

       from deltasigma import *
       import numpy as np
       import pylab as plt
       order = 4
       osr = 64
       nlev = 2
       f0 = 0.
       Hinf = 1.5
       form = 'CRFB'
       ntf = synthesizeNTF(order, osr, 2, Hinf, f0)
       a, g, b, c = realizeNTF(ntf, form)
       b = np.concatenate(( # Use a single feed-in for the input
                           np.atleast_1d(b[0]),
                           np.zeros((b.shape[0] - 1,))
                         ))
       ABCD = stuffABCD(a, g, b, c, form)
       DocumentNTF(ABCD, osr, f0)

    """
    if isinstance(arg1, np.ndarray):
        ABCD = arg1
        ntf, stf = calculateTF(ABCD)
    else:
        ntf = arg1
        stf = None

    fig = plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plotPZ(ntf, 'b', 6, showlist=False)
    plt.title('Poles and Zeros')
    plt.subplot(122)
    f = ds_freq(osr, f0, quadrature)
    z = np.exp(2j * np.pi * f)
    H = dbv(evalTF(ntf, z))
    plt.plot(f, H, 'b')

    if stf is not None:
        fig.suptitle('NTF and STF', fontsize=14)
        G = dbv(evalTF(stf, z))
        #plt.hold(True)
        plt.plot(f, G, 'm')
        #plt.hold(False)
    else:
        fig.suptitle('NTF', fontsize=14)

    f1, f2 = ds_f1f2(osr, f0, quadrature)
    NG0 = dbv(rmsGain(ntf, f1, f2))
    #plt.hold(True)
    plt.plot(np.array([f1, f2]), NG0*np.array([1, 1]), 'k', linewidth=3)

    if f0  ==  0:
        plt.text(0.5/osr, NG0, '  %.0fdB' % NG0, horizontalalignment = 'left',
                 verticalalignment = 'center')
    else:
        plt.text(f0, NG0 + 1, '%.0fdB' % NG0, horizontalalignment = 'center',
                 verticalalignment = 'bottom')
    msg = ' Inf-norm of H = %.2f\n 2-norm of H = %.2f' % (infnorm(ntf)[0], rmsGain(ntf, 0, 1))
    if quadrature:
        ING0 = dbv(rmsGain(ntf, - f1, - f2))
        plt.plot(-np.array([f1, f2]), ING0*np.array([1, 1]), 'k', linewidth=3)
        plt.text(-f0, ING0 + 1, '%.0fdB' % ING0, horizontalalignment = 'center',
                 verticalalignment = 'bottom')
        f_left = -0.5
    else:
        f_left = 0
    # variable 'f_left' used before assignment in DocumentNTF.m #REP
    if f0 < 0.25:
        plt.text(0.48, 0, msg, horizontalalignment = 'right', verticalalignment = 'top')
    else:
        plt.text(f_left, 0, msg, horizontalalignment = 'left', verticalalignment = 'top')
    plt.grid(True)
    y_bot = min(-80, np.round(NG0*1.1, -1))
    dx = 1./10 if quadrature else 1./20
    figureMagic(xRange=(f_left, 0.5), dx=dx, yRange=(y_bot, 15), dy=10)
    plt.ylabel('|H(f)| dB')
    plt.xlabel('Frequency ($1 \\rightarrow f_{s}$)')
    plt.title('Frequency Response')
    return


In [ ]:
DocumentNTF(ntf1, osr = 8)

In [ ]:
ntf2 = (np.array([-1. + 0.j, -1. + 0.j]), np.array([0. + 0.j, 0. + 0.j]), 1)

In [ ]:
DocumentNTF(ntf2, osr = 8, f0 = 0.5)

In [ ]:
ntf3 = (np.array([0. + 1.j, 0. - 1.j]), np.array([0. + 0.j, 0. + 0.j]), 1)

In [ ]:
DocumentNTF(ntf3, osr = 8)

In [ ]:
# -*- coding: utf-8 -*-
# _PlotExampleSpectrum.py
# Miscellaneous functions and stdlib wrappers for MATLAB functions
# that do not find a direct replacement in numpy/scipy.
# Copyright 2013 Giuseppe Venturini
# This file is part of python-deltasigma.
#
# python-deltasigma is a 1:1 Python replacement of Richard Schreier's
# MATLAB delta sigma toolbox (aka "delsigma"),  upon which it is heavily based.
# The delta sigma toolbox is (c) 2009,  Richard Schreier.
#
# python-deltasigma is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# LICENSE file for the licensing terms.

"""This module provides the PlotExampleSpectrum function.
"""

from __future__ import division

import numpy as np
import pylab as plt
from numpy.fft import fft, fftshift

from _calculateSNR import calculateSNR
from _circ_smooth import circ_smooth
from _dbp import dbp
from _dbv import dbv
from _ds_f1f2 import ds_f1f2
from _ds_hann import ds_hann
from _evalTF import evalTF
from _figureMagic import figureMagic
from _simulateDSM import simulateDSM
from _simulateQDSM import simulateQDSM
from _undbv import undbv


def PlotExampleSpectrum(ntf, M=1, osr=64, f0=0, quadrature=False):
    """Plot a spectrum suitable to exemplify the NTF performance.

    **Parameters:**

    ntf : scipy 'lti' object, tuple or ndarray
           The first argument may be one of the various supported
           representations for a (SISO) transfer function or an
           ABCD matrix. See :func:`evalTF` for a more detailed
           discussion.

    f0 : float, optional
         The center frequency. Normalized. Defaults to 0.

    M : int, optional
        M is defined as:

        .. math::

            M = n_{lev} - 1

        The number of quantizer levels (:math:`n_{lev}`) defaults to 2 and
        ``M`` defaults to 1.

    quadrature : boolean, optional
                 Whether the delta sigma modulator is a quadrature
                 modulator or not. Defaults to ``False``.

    .. plot::

        import pylab as plt
        from deltasigma import synthesizeNTF, PlotExampleSpectrum
        order = 3
        osr = 32
        f0 = 0.
        Hinf = 1.5
        ntf = synthesizeNTF(order, osr, 0, Hinf, f0)
        plt.figure(figsize=(12, 5))
        PlotExampleSpectrum(ntf, M=1, osr=osr, f0=f0)

    """
    f1, f2 = ds_f1f2(osr, f0, quadrature)
    delta = 2
    Amp = undbv(-3) # Test tone amplitude, relative to full-scale.
    # f below is the test tone frequency offset from f0, relative to bw.
    # (It will be adjusted to be an fft bin)
    f = 0.3
    N = 2**12
    f1_bin = np.round(f1*N)
    f2_bin = np.round(f2*N)
    fin = round(((1 - f)/2*f1 + (f + 1)/2*f2) * N)
    if not quadrature:
        t = np.arange(0, N).reshape((1, -1))
        u = Amp*M*np.cos((2*np.pi/N)*fin*t)
        v, _, xmax, y = simulateDSM(u, ntf, M+1)
    else:
        t = np.arange(0, N).reshape((1, -1))
        u = Amp*M*np.exp((2j*np.pi/N)*fin*t)
        v, _, xmax, y = simulateQDSM(u, ntf, M+1)
    window = ds_hann(N)
    NBW = 1.5/N
    spec0 = fft(v * window)/(M*N/4)
    if not quadrature:
        freq = np.linspace(0, 0.5, N//2 + 1)
        plt.plot(freq, dbv(spec0[:N//2 + 1]), 'c', linewidth=1)
        #plt.hold(True)
        spec_smoothed = circ_smooth(np.abs(spec0)**2., 16)
        plt.plot(freq, dbp(spec_smoothed[:N//2 + 1]), 'b', linewidth=3)
        Snn = np.abs(evalTF(ntf, np.exp(2j*np.pi*freq)))**2 * 2/12*(delta/M)**2
        plt.plot(freq, dbp(Snn*NBW), 'm', linewidth=1)
        f1_bin = int(f1_bin)
        f2_bin = int(f2_bin)
        snr = calculateSNR(spec0[f1_bin:f2_bin + 1], fin - f1_bin)
        msg = 'SQNR  =  %.1fdB\n @ A = %.1fdBFS & osr = %.0f\n' % \
              (snr, dbv(spec0[fin]), osr)
        if f0 < 0.25:
            plt.text(f0 + 1 / osr, - 15, msg, horizontalalignment='left',
                     verticalalignment='center')
        else:
            plt.text(f0 - 1 / osr, - 15, msg, horizontalalignment='right',
                     verticalalignment='center')
        plt.text(0.5, - 135, 'NBW = %.1e ' % NBW, horizontalalignment='right',
                 verticalalignment='bottom')
        figureMagic((0, 0.5), 1./16, None, (-140, 0), 10, None)
    else:
        spec0 = fftshift(spec0 / 2)
        freq = np.linspace(-0.5, 0.5, N + 1)
        freq = freq[:-1]
        plt.plot(freq, dbv(spec0), 'c', linewidth=1)
        plt.hold(True)
        spec_smoothed = circ_smooth(abs(spec0)**2, 16)
        plt.plot(freq, dbp(spec_smoothed), 'b', linewidth=3)
        Snn = abs(evalTF(ntf, np.exp(2j * np.pi * freq))) ** 2 * 2 / 12 * (delta / M) ** 2
        plt.plot(freq, dbp(Snn*NBW), 'm', linewidth=1)
        snr = calculateSNR(spec0[N//2 + f1_bin:N//2 + f2_bin + 1], fin - f1_bin)
        msg = 'SQNR  =  %.1fdB\n @ A = %.1fdBFS & osr = %.0f' % \
              (snr, dbv(spec0[N/2 + fin]), osr)
        if f0 >=  0:
            plt.text(f0 - 0.05, - 15, msg, horizontalalignment='right',
                     verticalalignment='bottom')
        else:
            plt.text(f0 + 0.05, - 15, msg, horizontalalignment='left',
                     verticalalignment='bottom')
        plt.text(-0.5, -135, ' NBW = %.1e' % NBW, horizontalalignment='left',
                 verticalalignment='bottom')
        figureMagic((-0.5, 0.5), 0.125, None, (-140, 0), 10, None)
    plt.xlabel('frequency')
    return


In [ ]:
PlotExampleSpectrum(ntf1, osr = 8)

In [ ]:
PlotExampleSpectrum(ntf2, osr = 8)

In [ ]:
PlotExampleSpectrum(ntf3, osr = 8)